In [ ]:
import librosa.display
import os
from src.config.config_defaults import get_default_config
from pathlib import Path
import pyrootutils
import IPython
from src.utils.utils_audio import plot_spectrograms

%load_ext autoreload
%autoreload 2

path_workdir: Path = Path(pyrootutils.find_root(search_from=os.curdir, indicator=".project-root"))
os.chdir(Path(path_workdir))

config = get_default_config()

def play_audio(audio, rate):
    IPython.display.display(IPython.display.Audio(data=audio, rate=rate))
    

In [ ]:
import torch
import torch.nn.functional
from src.config.argparse_with_config import ArgParseWithConfig
from src.features.chunking import (
    collate_fn_spectrogram,
    undo_image_chunking,
)
from src.utils.utils_audio import plot_spectrograms
from src.utils.utils_dataset import (
    get_example_val_sample,
    remove_rgb_channel,
)
from src.features.audio_to_spectrogram import MelSpectrogram
import matplotlib.pyplot as plt

audio = get_example_val_sample(config.sampling_rate)
original_spectrogram = librosa.feature.melspectrogram(
    y=audio,
    sr=config.sampling_rate,
    n_fft=config.n_fft,
    hop_length=config.hop_length,
    n_mels=config.n_mels,
)
transform = MelSpectrogram(
	sampling_rate=config.sampling_rate,
	hop_length=config.hop_length,
	n_fft=config.n_fft,
	n_mels=config.n_mels,
	image_size=config.image_size,
	spectrogram_augmentation=None,
	waveform_augmentation=None,
	max_num_width_samples=config.max_num_width_samples,
	normalize_audio=False,
	normalize_image=True
)
spectrogram = transform(audio)
out = collate_fn_spectrogram(
	[
		(spectrogram, torch.ones(11), torch.tensor([1])),
		(spectrogram, torch.ones(11), torch.tensor([3])),
	]
)
images, _, file_indices, _ = out
spectrogram_chunks = images[file_indices == 0]
spectrogram_reconstructed = undo_image_chunking(spectrogram_chunks, config.n_mels)
spectrogram_reconstructed = transform.undo(spectrogram_reconstructed).squeeze(0)

plot_spectrograms(
	[original_spectrogram, spectrogram_reconstructed],
	n_fft=config.n_fft,
	sampling_rate=config.sampling_rate,
	n_mels=config.n_mels,
	hop_length=config.hop_length,
	titles=["Original spectrogram", "Reconstructed spectrogram from chunking"]
)


spectrogram_chunks = transform.undo(spectrogram_chunks)
print(spectrogram_chunks.shape)

plot_spectrograms(spectrogram_chunks, n_fft=config.n_fft,
	sampling_rate=config.sampling_rate,
	n_mels=config.n_mels,
	hop_length=config.hop_length,
	titles=["Chunk"] * len(spectrogram_chunks))
